In [1]:
from src.imgModifier import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import ndimage
from PIL import Image
import scipy.signal
import glob
import pickle
import cv2
import os

### Pasamos videos a frames

- Para aumentar el numero imagenes convertimos videos de monedas a frames con la fn: videoToFrames(path)

In [2]:
'''
Ej:

videoToFrames('./Inputs/dataset_master/videos_2e/*.mp4')

'''

"\nEj:\n\nvideoToFrames('./Inputs/dataset_master/videos_2e/*.mp4')\n\n"

## Creamos el dataframe con las imagenes, labels, y paths originales

In [3]:
labels_t = ['10c','50c','20c','5c','2c','1e','1c','2e']
origin_path = "./Inputs/coins"

In [4]:
imagenes,img_label,img_path = readImages(origin_path,labels_t)

In [5]:
coins_original = pd.DataFrame(list(zip(img_label, imagenes, img_path)), columns =['label', 'image', 'path'])
print(coins_original.shape)
coins_original.head()


(2860, 3)


,label,image,path
0,10c,"[[[19, 68, 100], [52, 111, 143], [150, 220, 25...",Inputs/coins/10c/train_21_56.jpg
1,10c,"[[[221, 223, 223], [221, 223, 223], [221, 223,...",Inputs/coins/10c/test_18_24.jpg
2,10c,"[[[65, 81, 104], [47, 63, 86], [34, 50, 73], [...",Inputs/coins/10c/IMG_4187_22copia.jpg
3,10c,"[[[65, 97, 132], [51, 83, 118], [43, 74, 107],...",Inputs/coins/10c/IMG_4207_17.jpg
4,10c,"[[[68, 75, 130], [60, 70, 124], [66, 75, 132],...",Inputs/coins/10c/IMG_4187_23copia.jpg


In [6]:
coins_original['image'].iloc[0].shape

(150, 150, 3)

In [7]:
print(coins_original['label'].unique())
print(coins_original.shape)
print(coins_original['label'].value_counts())

['10c' '50c' '20c' '5c' '2c' '1e' '1c' '2e']
(2860, 3)
5c     385
1c     382
2c     378
50c    355
2e     344
10c    343
20c    342
1e     331
Name: label, dtype: int64


## Creamos segundo df con los arrays de las imagenes originales (aplicando el resize (70x70) ) y sus labels 

In [8]:
coins = pd.DataFrame()

In [9]:
coins['image'] = coins_original['image'].apply(transfImg)

In [10]:
coins = coins.join(coins_original['label'])

In [11]:
display(coins.head())
print(coins.shape)


,image,label
0,"[[[27, 81, 110], [94, 165, 195], [30, 85, 117]...",10c
1,"[[[227, 229, 229], [224, 226, 226], [214, 216,...",10c
2,"[[[58, 74, 97], [32, 47, 71], [28, 44, 67], [5...",10c
3,"[[[60, 92, 127], [44, 75, 108], [65, 93, 123],...",10c
4,"[[[63, 71, 126], [53, 65, 121], [61, 76, 136],...",10c


(2860, 2)


In [12]:
coins['image'].iloc[0].shape

(70, 70, 3)

## Creamos más dfs con los arrays rotados para tener más datos  y los concatenamos al anterior df (coins)

In [13]:
'''
# Rota los arrays x grados
def rotateConcat(coins_original, coins, grados):
    coins_gr = pd.DataFrame()
    coins_gr['image'] = coins_original['image'].apply(lambda x: rotAndTransfImg(x,grados))
    coins_gr = coins_gr.join(coins_original['label'])
    coins = pd.concat([coins,coins_gr], ignore_index=True)
    print(coins.shape)
    return coins
'''

"\n# Rota los arrays x grados\ndef rotateConcat(coins_original, coins, grados):\n    coins_gr = pd.DataFrame()\n    coins_gr['image'] = coins_original['image'].apply(lambda x: rotAndTransfImg(x,grados))\n    coins_gr = coins_gr.join(coins_original['label'])\n    coins = pd.concat([coins,coins_gr], ignore_index=True)\n    print(coins.shape)\n    return coins\n"

In [14]:

print(coins['label'].unique())
print(coins.shape)
print(coins['label'].value_counts())

['10c' '50c' '20c' '5c' '2c' '1e' '1c' '2e']
(2860, 2)
5c     385
1c     382
2c     378
50c    355
2e     344
10c    343
20c    342
1e     331
Name: label, dtype: int64


In [15]:
grados = [e for e in range(1,360,10)]
len(grados)

36

In [16]:
# Aplicamos la fn para rotar y concatenar los arrays al df
for grado in grados:
    coins = rotateConcat(coins_original, coins, grado)

(5720, 2)
(8580, 2)
(11440, 2)
(14300, 2)
(17160, 2)
(20020, 2)
(22880, 2)
(25740, 2)
(28600, 2)
(31460, 2)
(34320, 2)
(37180, 2)
(40040, 2)
(42900, 2)
(45760, 2)
(48620, 2)
(51480, 2)
(54340, 2)
(57200, 2)
(60060, 2)
(62920, 2)
(65780, 2)
(68640, 2)
(71500, 2)
(74360, 2)
(77220, 2)
(80080, 2)
(82940, 2)
(85800, 2)
(88660, 2)
(91520, 2)
(94380, 2)
(97240, 2)
(100100, 2)
(102960, 2)
(105820, 2)


In [17]:
# Comprobamos que todos los arrays sean de 70x70x3
for im in coins['image']:
    if im.shape != (70, 70, 3):
        print('¡ojo!', im.shape)

In [31]:
# Guardamos nuestro df en un archivo pickle:

coins.to_pickle('./Output/data/coins_arrays_8label.pkl')  

In [27]:
coins['label'].value_counts()

5c     14245
1c     14134
2c     13986
50c    13135
2e     12728
10c    12691
20c    12654
1e     12247
Name: label, dtype: int64

In [19]:
coins_4lab = coins.copy()

In [20]:
coins_4lab['label'].unique()

array(['10c', '50c', '20c', '5c', '2c', '1e', '1c', '2e'], dtype=object)

In [21]:
coins_4lab['label'] = coins_4lab['label'].replace({'10c':'au','50c':'au','20c':'au', '1c':'cu','2c':'cu','5c':'cu'})

In [24]:
coins_4lab['label'].unique()

array(['au', 'cu', '1e', '2e'], dtype=object)

In [25]:
coins4dumm = pd.get_dummies(coins_4lab['label'])
coins4dumm['gt'] = coins4dumm.apply(lambda x: pd.Series([x.values]), axis=1)
coins_4labels=coins_4lab.join(coins4dumm)


In [26]:
coins_4labels.head()

,image,label,1e,2e,au,cu,gt
0,"[[[27, 81, 110], [94, 165, 195], [30, 85, 117]...",au,0,0,1,0,"[0, 0, 1, 0]"
1,"[[[227, 229, 229], [224, 226, 226], [214, 216,...",au,0,0,1,0,"[0, 0, 1, 0]"
2,"[[[58, 74, 97], [32, 47, 71], [28, 44, 67], [5...",au,0,0,1,0,"[0, 0, 1, 0]"
3,"[[[60, 92, 127], [44, 75, 108], [65, 93, 123],...",au,0,0,1,0,"[0, 0, 1, 0]"
4,"[[[63, 71, 126], [53, 65, 121], [61, 76, 136],...",au,0,0,1,0,"[0, 0, 1, 0]"


In [30]:
coins_4labels.to_pickle('./Output/data/coins_dum_4labels.pkl')